In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from IPython.display import display 

import spacy

In [3]:
folder = os.path.join(os.getcwd(), "data", "transcripts")
files = [os.path.join(root, f) for root, _, files in os.walk(folder) for f in files if f.endswith(".docx")]
        

In [4]:
from src.pipelinelib.querying import Parser, Queryable

nlp = spacy.load("de_core_news_sm", disable=["ner", "parser"])
parser = Parser(nlp=nlp, metadata_path="./data/transcripts/Kopie von Transkriptionspaare_Daten.xls")

parser.read_from_files(files)

=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 182_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 27_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 60_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 47_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 58_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 29_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 87_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 81_T1_IM_FW_corrected.docx ===
=== Parser: reading from /home/alexander/Projects/sigmund/data/transcripts/Paar 105_T1_IM_FW_corrected.docx ===

In [5]:
display(parser.frame)

,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,1,0,182,B,M,False,None,Hm?
1,0,2,1,182,A,W,False,None,Du solltest doch nicht nachgucken.
2,0,3,2,182,B,M,False,None,Sag nur ``Ich darf den schönen Mädchen nicht n...
3,0,4,3,182,A,W,False,None,"Doch, darfst du."
4,0,6,4,182,B,M,False,None,Also.
...,...,...,...,...,...,...,...,...,...
114,9,87,114,138,B,M,True,W,"Doch, das ist schon was Besonderes, ich hab je..."
115,9,88,115,138,A,W,True,W,Hm .
116,9,88,116,138,A,W,True,W,Ja es gibt viele Erinnerungen die irgendwie di...
117,9,88,117,138,A,W,True,W,Es ist interessant.


In [6]:
from src.pipelinelib.querying import Parser, Queryable
from src.pipelinelib.text_body import TextBody

queryable = Queryable.from_parser(parser)

In [7]:
document_df = queryable.execute(level=TextBody.DOCUMENT)
display(document_df)

,document_id,couple_id,is_depressed_group,depressed_person,text
0,0,182,False,None,Hm?. Du solltest doch nicht nachgucken.. Sag n...
1,1,27,True,W,"Positive Gefühle.. Ja, positive Gefühle können..."
2,2,60,True,W,"Wenn´s positiv geht, dann wird es nicht laut, ..."
3,3,47,True,W,Erzähl mir was.. Also wir müssen die Fragen je...
4,4,58,False,None,Soll ich anfangen?. Ich hab mir was überlegt.....
5,5,29,False,None,"Gut, fangen wir mal an.. Ja, fang du mal an.. ..."
6,6,87,False,None,"Wie lang?. 10 Minuten. Okay, dann fang mal an...."
7,7,81,False,None,Ich dachte wir reden darüber wie wir die erste...
8,8,105,True,W,Jetzt sollen wir uns unterhalten?. Genau.. Al...
9,9,138,True,W,"Am Ammersee.. Dann machen wir es so.. Da, wo m..."


In [8]:
paragraph_df = queryable.execute(level=TextBody.PARAGRAPH)
display(paragraph_df)

,document_id,paragraph_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,1,182,B,M,False,None,Hm?
1,0,2,182,A,W,False,None,Du solltest doch nicht nachgucken.
2,0,3,182,B,M,False,None,Sag nur ``Ich darf den schönen Mädchen nicht n...
3,0,4,182,A,W,False,None,"Doch, darfst du."
4,0,6,182,B,M,False,None,Also.
...,...,...,...,...,...,...,...,...
750,9,85,138,B,M,True,W,Okay und das ist dann wenn du so ein Foto schi...
751,9,86,138,A,W,True,W,"Ja, empfindest du das nicht so?"
752,9,87,138,B,M,True,W,"Doch, das ist schon was Besonderes, ich hab je..."
753,9,88,138,A,W,True,W,Hm .. Ja es gibt viele Erinnerungen die irgend...


In [9]:
sentence_df = queryable.execute(level=TextBody.SENTENCE)
display(sentence_df)

queryable.execute(level=TextBody.SENTENCE)

,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,1,0,182,B,M,False,None,Hm?
1,0,2,1,182,A,W,False,None,Du solltest doch nicht nachgucken.
2,0,3,2,182,B,M,False,None,Sag nur ``Ich darf den schönen Mädchen nicht n...
3,0,4,3,182,A,W,False,None,"Doch, darfst du."
4,0,6,4,182,B,M,False,None,Also.
...,...,...,...,...,...,...,...,...,...
114,9,87,114,138,B,M,True,W,"Doch, das ist schon was Besonderes, ich hab je..."
115,9,88,115,138,A,W,True,W,Hm .
116,9,88,116,138,A,W,True,W,Ja es gibt viele Erinnerungen die irgendwie di...
117,9,88,117,138,A,W,True,W,Es ist interessant.


,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,1,0,182,B,M,False,None,Hm?
1,0,2,1,182,A,W,False,None,Du solltest doch nicht nachgucken.
2,0,3,2,182,B,M,False,None,Sag nur ``Ich darf den schönen Mädchen nicht n...
3,0,4,3,182,A,W,False,None,"Doch, darfst du."
4,0,6,4,182,B,M,False,None,Also.
...,...,...,...,...,...,...,...,...,...
114,9,87,114,138,B,M,True,W,"Doch, das ist schon was Besonderes, ich hab je..."
115,9,88,115,138,A,W,True,W,Hm .
116,9,88,116,138,A,W,True,W,Ja es gibt viele Erinnerungen die irgendwie di...
117,9,88,117,138,A,W,True,W,Es ist interessant.


In [10]:
pgs = queryable.by_couple_id(couple_id=27) \
    .is_depressed(d=False) \
    .execute(level=TextBody.DOCUMENT)
display(pgs)

,document_id,couple_id,is_depressed_group,depressed_person,text


'\npipeline  = Pipeline(queryable=queryable)\npipeline.add_component(Tokenizer())\ndisplay(pipeline.execute())\n'

In [35]:
from src.sigmund.preprocessing.words import Tokenizer, Stemmer, Lemmatizer
from src.sigmund.features.tfidf import FeatureTFIDF
from src.pipelinelib.pipeline import Pipeline
from src.sigmund.classification.naive_bayes import NaiveBayes
from src.sigmund.classification.merger import FeatureMerger
from src.sigmund.classification.linear_discriminant_analysis import LinearDiscriminantAnalysisClassifier
from src.sigmund.features.liwc import Liwc
from src.sigmund.features.vocabulary_size import VocabularySize
from src.sigmund.classification.pca import PCAReduction
from src.sigmund.extensions import *

pipeline = Pipeline(queryable=queryable)
pipeline.add_component(Tokenizer())
pipeline.add_component(Stemmer())
pipeline.add_component(Lemmatizer())
pipeline.add_component(FeatureTFIDF(white_list=[
    'ja', 'auch', 'wenn', 'also', 'werden', 'schon', 'wir', # high in depressed group
    'und', 'haben', 'du', 'sehr'])), #'so', 'wirkl  ich', 'ich', 'gerne', 'weil']))
#pipeline.add_component(FeatureTFIDF())
pipeline.add_component(Liwc())
#pipeline.add_component(FeatureMerger()) #[TFIDF_DOCUMENT]))

pipeline.add_component(NaiveBayes(inputs=[TFIDF_DOCUMENT_MF], output=CLASSIFICATION_NAIVE_BAYES_TFIDF, voting=False))

pipeline.add_component(NaiveBayes(inputs=[LIWC_DOCUMENT_MF], output=CLASSIFICATION_NAIVE_BAYES_LIWC, voting=False))

pipeline.add_component(NaiveBayes(inputs=[CLASSIFICATION_NAIVE_BAYES_TFIDF, CLASSIFICATION_NAIVE_BAYES_LIWC], output=CLASSIFICATION_NAIVE_BAYES_VOTING, voting=True))


pipeline.execute(visualise=True)

=== Starting pipeline with ['Tokenizer', 'Stemmer', 'Lemmatizer', 'FeatureTFIDF', 'Liwc', 'NaiveBayes', 'NaiveBayes', 'NaiveBayes'] ===
Executing Tokenizer
Executing Stemmer
Executing Lemmatizer
Executing FeatureTFIDF
Executing Liwc
Executing NaiveBayes


,couple_id,is_depressed_group,ja,auch,wenn,also,werden,schon,wir,und,haben,du,sehr
0,182,False,0.298184,0.156191,0.063896,0.021299,0.014199,0.023329,0.092295,0.198789,0.106494,0.191689,0.042621
1,27,True,0.294945,0.196630,0.163858,0.024579,0.049157,0.000000,0.180244,0.180244,0.057350,0.221209,0.019674
2,60,True,0.257762,0.171841,0.121721,0.028640,0.093081,0.164692,0.085921,0.157521,0.078760,0.264922,0.008597
3,47,True,0.357365,0.299189,0.116351,0.074797,0.041554,0.027309,0.157905,0.199459,0.058176,0.132973,0.029936
4,58,False,0.108745,0.163118,0.027186,0.006797,0.040780,0.029777,0.156321,0.244677,0.142728,0.224287,0.146888
5,29,False,0.261638,0.098114,0.039246,0.078491,0.032705,0.071644,0.045787,0.235474,0.143901,0.215851,0.054975
6,87,False,0.387505,0.182681,0.155002,0.033215,0.033215,0.036380,0.105180,0.132859,0.121787,0.359826,0.046527
7,81,False,0.141206,0.111253,0.077021,0.004279,0.021395,0.018747,0.179716,0.308085,0.201111,0.209669,0.000000
8,105,True,0.249690,0.168908,0.102813,0.066094,0.066094,0.032175,0.124845,0.139532,0.176252,0.330472,0.000000
9,138,True,0.446558,0.195369,0.055820,0.050238,0.016746,0.091710,0.150713,0.167459,0.044656,0.061402,0.053617


'Predictions on the test set'

,couple_id,is_depressed_group,predicted
8,105,1,1
1,27,1,1
5,29,0,1
0,182,0,1
7,81,0,1


'Cross-valiation'

...

,couple_id,is_depressed_group,is_depressed_group,predicted
0,182,False,0,0
1,27,True,1,1
2,60,True,1,0
3,47,True,1,0
4,58,False,0,0
5,29,False,0,0
6,87,False,0,0
7,81,False,0,0
8,105,True,1,0
9,138,True,1,1


'Accuracy on test set: 0.4'

'Accuracy with cross-valiation: [1.  0.5 0.5 0.5 1. ] | mean = 0.7'

=== Finished pipeline execution ===


 0.000000  
 6  0.116709  0.147832  0.108929  0.326786  0.051292  
 7  0.077820  0.140077  0.155641  0.249025  0.000000  
 8  0.215067  0.143378  0.071689  0.047793  0.031506  
 9  0.059748  0.179245  0.059748  0.149371  0.078776  ,
 tfidf_document_f(feature):    couple_id        ja      auch      wenn      also    werden     schon  \
 0         27  0.290161  0.152717  0.198532  0.033454  0.066909  0.000000   
 1         29  0.159569  0.087763  0.039892  0.096128  0.043694  0.094673   
 2         47  0.387110  0.294204  0.123875  0.118722  0.033920  0.061246   
 3         58  0.138962  0.238220  0.059555  0.021744  0.021744  0.000000   
 4         60  0.169254  0.199122  0.139386  0.032715  0.109050  0.249407   
 5         81  0.123322  0.147986  0.098657  0.000000  0.009005  0.021679   
 6         87  0.536416  0.150867  0.100578  0.018361  0.036721  0.066304   
 7        105  0.188466  0.175902  0.050258  0.027524  0.082572  0.000000   
 8        138  0.470462  0.190187  0.060059  0.